## Preprocessing Dataset

In [ ]:
training_dataset_path = "/content/drive/MyDrive/Linguistic Causation/Datasets/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT"

In [ ]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
all_lines = [] 
with open(training_dataset_path) as f:
  all_lines = f.readlines() 

all_lines

In [ ]:
# TODO: 
# For sentences with no verbs in between use the other verbs in the sentence 
# Use the order of cause and effect (if e1 causes e2 or vice versa) for training model too 

# An array that contains the array [NP1, V1, NP2] where NP1 is noun phrase 1, V1 is the first verb 
# encountered between the two noun phrases, and NP2 is noun phrase 2 
positive_arr = [] 
negative_arr = [] 

failed_counter = 0 
total_positive = 0 
total_negative = 0 
positive_no_verb = 0
negative_no_verb = 0 

for counter, line in enumerate(all_lines):
  # if(counter == 5000):
    # break
  print("{}/{}".format(counter, len(all_lines)))
  if(line[0].isdigit()):
    failed = False 
    try:
      result = re.search("<e1>.*</e2>", line).group(0)
    except:
      failed_counter += 1 

    if(not failed):
      np1 = re.search("<e1>(.+?)</e1>", result).group(1) 
      np2 = re.search("<e2>(.+?)</e2>", result).group(1) 

      result = result.replace("<e1>", "").replace("</e1>","").replace("<e2>","").replace("</e2>","")
      doc = nlp(result) 

      verbs = []
      for token in doc:
          if token.pos_ == "VERB":
            if(token.text != np1 and token.text != np2):
              verbs.append((token.text))
              # only add first verb 
              break 


      to_append = [] 
      to_append.append(np1)
      to_append = to_append + verbs
      to_append.append(np2) 
      # to_append.append(line) <- order of e1, e2 causality 

      # print("---")
      # print(to_append) 
      # print("---")
      
      no_verb = False
      if(len(verbs) == 0): 
        no_verb = True 

      if(all_lines[counter + 1].startswith("Cause-Effect")):
        total_positive += 1
        if(no_verb):
          positive_no_verb += 1 
        else:
          positive_arr.append(to_append)
      else:
        total_negative += 1
        if(no_verb):
          negative_no_verb += 1 
        else:
          negative_arr.append(to_append)

Streaming output truncated to the last 5000 lines.
27000/32000
27001/32000
27002/32000
27003/32000
27004/32000
27005/32000
27006/32000
27007/32000
27008/32000
27009/32000
27010/32000
27011/32000
27012/32000
27013/32000
27014/32000
27015/32000
27016/32000
27017/32000
27018/32000
27019/32000
27020/32000
27021/32000
27022/32000
27023/32000
27024/32000
27025/32000
27026/32000
27027/32000
27028/32000
27029/32000
27030/32000
27031/32000
27032/32000
27033/32000
27034/32000
27035/32000
27036/32000
27037/32000
27038/32000
27039/32000
27040/32000
27041/32000
27042/32000
27043/32000
27044/32000
27045/32000
27046/32000
27047/32000
27048/32000
27049/32000
27050/32000
27051/32000
27052/32000
27053/32000
27054/32000
27055/32000
27056/32000
27057/32000
27058/32000
27059/32000
27060/32000
27061/32000
27062/32000
27063/32000
27064/32000
27065/32000
27066/32000
27067/32000
27068/32000
27069/32000
27070/32000
27071/32000
27072/32000
27073/32000
27074/32000
27075/32000
27076/32000
27077/32000
27078/32000
2

In [ ]:
print("Sentences failed to parse: {}".format(failed_counter)) 
print("Total Positive: {}".format(total_positive))
print("No Verb Positive: {}".format(positive_no_verb))
print("Total Usable Positive: {}".format(total_positive - positive_no_verb))
print("Total Negative: {}".format(total_negative))
print("No Verb Negative: {}".format(negative_no_verb))
print("Total Usable Negative: {}".format(total_negative - negative_no_verb))

Sentences failed to parse: 0
Total Positive: 1003
No Verb Positive: 282
Total Usable Positive: 721
Total Negative: 6997
No Verb Negative: 3191
Total Usable Negative: 3806


In [ ]:
print(len(positive_arr))
print(len(negative_arr))

721
3806


In [ ]:
positive_arr[0:10]

[['inflammation', 'caused', 'infection'],
 ['burst', 'caused', 'pressure'],
 ['singer', 'performed', 'commotion'],
 ['Suicide', 'leading', 'death'],
 ['women', 'caused', 'accident'],
 ['Calluses', 'caused', 'skin abnormality'],
 ['meteor shower', 'caused', 'comet'],
 ['recurrence', 'arose', 'radiation'],
 ['outbreak', 'caused', 'vaccine'],
 ['fire', 'caused', 'fuel']]

In [ ]:
negative_arr[0:10]

[['child', 'wrapped', 'cradle'],
 ['author', 'uses', 'disassembler'],
 ['People', 'moving', 'downtown'],
 ['lawsonite', 'contained', 'platinum crucible'],
 ['solvent', 'pipetted', 'flask'],
 ['essays', 'collected', 'volume'],
 ['composer', 'sunk', 'oblivion'],
 ['citation', 'explaining', 'reasons'],
 ['networks', 'moved', 'high-definition broadcast'],
 ['call', 'remind', 'bill']]

## Getting Word Embeddings

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
glove_file = datapath('/content/drive/MyDrive/Linguistic Causation/Datasets/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [ ]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [ ]:
def transform_to_embeddings(arr):
  new_arr = [] 
  for sentence in arr:
      try:
        to_append = [model[token] for token in sentence] 
        new_arr.append(to_append) 
      except KeyError as e:
        print(e) 
      

  return new_arr

In [ ]:
# TO DO: 
# When Creating Embeddings below keep largetst NP as concent in WordNet like in Girju et al. 

In [ ]:
# OOV (Out of vocab words) -> Byte pair encoding 

In [ ]:
# Creating Embedding for Positive Examples
# Lower all the words when searching for word embedding 

import numpy as np 
new_positive_arr = transform_to_embeddings(positive_arr)
print(np.shape(positive_arr))
print(np.shape(new_positive_arr))
print("Examples lost: {}".format(len(positive_arr) - len(new_positive_arr)))

"word 'Suicide' not in vocabulary"
"word 'Calluses' not in vocabulary"
"word 'meteor shower' not in vocabulary"
"word 'Leakage' not in vocabulary"
"word 'Contractions' not in vocabulary"
"word 'Cardiovascular disease' not in vocabulary"
"word 'Diabetes mellitus' not in vocabulary"
"word 'Preeclampsia' not in vocabulary"
"word 'Alcohol' not in vocabulary"
"word 'Sodium' not in vocabulary"
"word 'Convulsions' not in vocabulary"
"word 'Fog' not in vocabulary"
"word 'pain killers' not in vocabulary"
"word 'Weak ligaments' not in vocabulary"
"word 'mycoses' not in vocabulary"
"word 'Swelling' not in vocabulary"
"word 'peakeffect' not in vocabulary"
"word 'Sun' not in vocabulary"
"word 'shoulder problems' not in vocabulary"
"word 'Smoke' not in vocabulary"
"word 'Germs' not in vocabulary"
"word 'boiling water' not in vocabulary"
"word 'Signs' not in vocabulary"
"word 'Discussion' not in vocabulary"
"word 'grinding noise' not in vocabulary"
"word 'Poverty' not in vocabulary"
"word 'tectonic f

In [ ]:
# Creating Embedding for Negative Examples

import numpy as np 
new_negative_arr = transform_to_embeddings(negative_arr)
print(np.shape(negative_arr))
print(np.shape(new_negative_arr))
print("Examples lost: {}".format(len(negative_arr) - len(new_negative_arr)))

"word 'People' not in vocabulary"
"word 'lawsonite' not in vocabulary"
"word 'pipetted' not in vocabulary"
"word 'high-definition broadcast' not in vocabulary"
"word 'rural history' not in vocabulary"
"word 'Newspapers' not in vocabulary"
"word 'Adults' not in vocabulary"
"word 'causal part' not in vocabulary"
"word 'double bags' not in vocabulary"
"word 'Chemical weapons' not in vocabulary"
"word 'bacterial aerosol' not in vocabulary"
"word 'People' not in vocabulary"
"word 'membrane tubing' not in vocabulary"
"word 'theological survey' not in vocabulary"
"word 'Positions' not in vocabulary"
"word 'Governments' not in vocabulary"
"word 'nearby countries' not in vocabulary"
"word 'Mobile' not in vocabulary"
"word 'indenter' not in vocabulary"
"word 'Cruisers' not in vocabulary"
"word 'oral culture' not in vocabulary"
"word 'Feedback' not in vocabulary"
"word 'soap operas' not in vocabulary"
"word 'television network' not in vocabulary"
"word 'cutting board' not in vocabulary"
"word 'fi

## Training Decision Tree

In [ ]:
print(len(new_positive_arr))
print(len(new_negative_arr))

593
3205


In [ ]:
# Concatenating positive and negative array word embeddings into one giant 
# array to use as features for decision tree 
features = new_positive_arr + new_negative_arr
print(np.shape(features))

# Making targets on the feature is simply setting a new array with 
# True values as the length of positive array and False values as length 
# of negative array 
targets = [True]  * len(new_positive_arr) + [False] * len(new_negative_arr)
print(np.shape(targets))

(3798, 3, 100)
(3798,)


In [ ]:
features[0:len(new_positive_arr)] == new_positive_arr

True

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.20)

In [ ]:
np.array(X_train).shape

(3038, 3, 100)

In [ ]:
np.array(y_train).shape

(3038,)

In [ ]:
np.array(X_train[0]).shape

(3, 100)

In [ ]:
# Converting 3D Array -> 2D Array as is required for sklearn fit functions 
# uses simple matrix multiplicatino to do this 
nsamples, nx, ny = np.array(X_train).shape
X_train_dataset = np.array(X_train).reshape((nsamples,nx*ny))

nsamples, nx, ny = np.array(X_test).shape
X_test_dataset = np.array(X_test).reshape((nsamples,nx*ny))

In [ ]:
# new 2D Array for fit function 
X_train_dataset.shape

(3038, 300)

In [ ]:
# Fottomg <pde; 
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_dataset, y_train)

In [ ]:
y_pred = clf.predict(X_test_dataset)

In [ ]:
# Evaluating Model
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[598  30]
 [ 28 104]]
              precision    recall  f1-score   support

       False       0.96      0.95      0.95       628
        True       0.78      0.79      0.78       132

    accuracy                           0.92       760
   macro avg       0.87      0.87      0.87       760
weighted avg       0.92      0.92      0.92       760



## Validation

In [ ]:
np.array(X_test[15]).shape

(3, 100)

In [ ]:
np.array(features).shape

(3798, 3, 100)

In [ ]:
len(X_test)

760

In [ ]:
correct = 0
answer_dict = {} 

for x in range(len(X_test)):
  test_index = x
  orig_index = np.where(np.all(np.all(np.array(X_test[test_index]) == np.array(features), 1), 1) == True)[0][0]
  orig_orig_index = len(features) - orig_index

  if(orig_orig_index < len(new_positive_arr)):
    orig_sentence = positive_arr[orig_orig_index]
    orig_class = True
  else:
    orig_sentence = negative_arr[orig_orig_index]
    orig_class = False

  if(orig_class == y_pred[test_index]):
    correct += 1
    answer_dict[x] = [orig_sentence, True]
  else:
    answer_dict[x] = [orig_sentence, False]
  # print(orig_sentence)
  # print(orig_class)
  # print(y_pred[test_index])

print("{}/{}".format(correct, len(X_test)))

529/760


In [ ]:
answer_dict

{0: [['discussion', 'pointed', 'people'], True],
 1: [['engine', 'powered', 'model'], False],
 2: [['woman', 'took', 'taxi'], True],
 3: [['tier', 'made', 'towels'], True],
 4: [['movie', 'made', 'monkey'], True],
 5: [['answer', 'solicited', 'mouth'], True],
 6: [['road', 'starts', 'intersection'], True],
 7: [['girl', 'loving', 'family'], True],
 8: [['convergence', 'caused', 'circulation'], False],
 9: [['houses', 'built', 'way'], True],
 10: [['Suspects', 'brought', 'custody'], True],
 11: [['shortage', 'cause', 'anemia'], False],
 12: [['bag', 'delivered', 'market'], False],
 13: [['file', 'contains', 'settings'], True],
 14: [['aspects', 'indicated', 'signals'], True],
 15: [['stars', 'left', 'course'], True],
 16: [['assembly', 'comprising', 'knob'], True],
 17: [['programme', 'detailed', 'history'], True],
 18: [['burner', 'works', 'herbs'], True],
 19: [['cycle', 'shortened', 'months'], True],
 20: [['institution', 'provides', 'deposit'], False],
 21: [['devices', 'added', 'ph

In [ ]:
test_index = 705

In [ ]:
# In order of new_positive_arr + new_negative_arr
orig_index = np.where(np.all(np.all(np.array(X_test[test_index]) == np.array(features), 1), 1) == True)[0][0]

In [ ]:
orig_index

1652

In [ ]:
features[orig_index] == X_test[test_index]

True

In [ ]:
print(len(new_positive_arr))
print(len(new_negative_arr))
print(len(features))

593
3205
3798


In [ ]:
orig_orig_index = len(features) - orig_index

In [ ]:
if(orig_orig_index < len(new_positive_arr)):
  orig_sentence = positive_arr[orig_orig_index]
  orig_class = "TRUE"
else:
  orig_sentence = negative_arr[orig_orig_index]
  orig_class = "FALSE"

In [ ]:
print(orig_sentence)
print(orig_class)
print(y_pred[test_index])

['photograph', 'enclosed', 'letter']
FALSE
True


## Miscellaneous

In [ ]:
import re 
s = "The current view is that the chronic <e1>inflammation</e1> in the distal part of the stomach caused by Helicobacter pylori <e2>infection</e2> results in an increased acid production from the non-infected upper corpus region of the stomach."
result = re.search("<e1>.*</e2>", s).group(0)
print(result)

np1 = re.search("<e1>(.+?)</e1>", result).group(1) 

np2 = re.search("<e2>(.+?)</e2>", result).group(1) 


result = result.replace("<e1>", "").replace("</e1>","").replace("<e2>","").replace("</e2>","")

print(np1)
print(np2) 
print(result)

<e1>inflammation</e1> in the distal part of the stomach caused by Helicobacter pylori <e2>infection</e2>
inflammation
infection
inflammation in the distal part of the stomach caused by Helicobacter pylori infection


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(result) 

words = []
for token in doc:
    # if token.pos_ == "NOUN" or token.pos_ == "VERB":
    if token.pos_ == "VERB":
        start = token.idx  # Start position of token
        end = token.idx + len(token)  # End position = start + len(token)
        # words.append((token.text, start, end, token.pos_))
        words.append((token.text))

print(words)

['caused']
